# Linear system response
A linear system can be described by its impulse response, or equally by its *z*-transform.  It can be useful to be able to relate the *z*-transform to the frequency and phase response of the linear system.

### Preamble
Start by importing the Python libraries that we will require

### Import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sps

And define a function that will return true if running in a Jupyter Notebook

In [ ]:
def is_jupyter():
    """Return true if running in a Jupyter Notebook"""
    try:
        if get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
            return True
        else:
            return False
    except: 
        return False

### User specified parameters
The following parameters can be specified. 

Parameter | Meaning
--------- | -------
<code>b</code>| The numerator polynomial for the transfer function
<code>a</code> | The denominator polynomial for the transfer function

In [ ]:
# Define the filter coefficients for Example 5.2.1
# Note that this is defined in terms of positive powers of z
b = [1, 1, 0]
a = [1, 0.1, -0.2]

### Plotting functions
First we define a plotting function for a traditional cartesian plot with frequency along the x-axis.

In [ ]:
def plot_response(x, y, ylim, ylabel, title, name):
    """
       Plot the response of normalised frequency
       
       Parameters:
                  x (array-like): The x-positions of plot. 
                  y (array-like): The y-positions of plot. 
                  ylim ([left, right]): Set the ylim to left, right.
                  ylabel (str): The label for the y-axis.
                  title (str): The title of figure.
                  name (str): Name used to save figure.
    """
    
    # Create the figure
    plt.figure(figsize = (16, 8))
    
    # Enlarge figure label and axis size
    plt.rcParams.update({'font.size': 16})
    
    # Plot the response
    plt.plot(x, y)
    
    # Tidy up the plot to control axes sizes and labels
    plt.ylim(ylim)
    plt.xlim([0,np.pi])
    plt.xticks(np.linspace(0, np.pi, 5),
               ['0','$\pi/4$','$\pi/2$','$3\pi/4$','$\pi$'])
    plt.xlabel('Normalized Frequency (rad/sample)')
    plt.ylabel(ylabel)
    plt.grid(True)
    plt.title(title)
    
    # Save figure in python or ipython system
    if not is_jupyter(): plt.savefig(name)

We also need a function for producing plots in the *z*-domain.  It will have axes for real and imaginary, as well as a unit circle marked on each plot.  It then adds in markers for each pole and each zero, which are specified as a list of complex values.

In [ ]:
def pz_plot(z, p):
    """
       Plot the poles and zeros.
       
       INPUT:
           z (complex number): Represent zeros.
           p (complex number): Represent poles.
           
       RETURN:
           ax: Current Axes instance on the current figure.
           
    """
    
    # Create a plot figure
    plt.figure(figsize = (16, 8))
    plt.rcParams.update({'font.size': 16})
    plt.ion()
    ax = plt.gca()
    
    # Set two axes with same standard tick length
    ax.set_aspect(1)
    
    # Plot the unit circle 
    circle1 = plt.Circle((0, 0), 1, 
                         color = '#87cefa', 
                         fill = False,
                         linestyle = 'dotted')
    
    # Plot the two straight lines perpendicular to 
    # each other through the origin
    ax.plot(np.linspace(-2, 2, 100), 
            np.zeros(100), 
            color = '#87cefa', 
            linestyle = 'dotted')
    ax.plot(np.zeros(100),
            np.linspace(-1, 1, 100),  
            color = '#87cefa',
            linestyle = 'dotted')
    
    # Plot zeros 
    for i in range(len(z)):
        ax.plot(np.real(z[i]),
                np.imag(z[i]), 
                'bo', 
                markerfacecolor = 'none')
                
    # Plot poles 
    for i in range(len(p)):
        ax.plot(np.real(p[i]), 
                np.imag(p[i]), 
                'bx')
    
    # Tidy up the plot to control axes sizes and labels
    ax.set_xticks(np.linspace(-2, 2, 5))
    ax.set_yticks(np.linspace(-1, 1, 5))
    ax.set_xlabel('Real Part')
    ax.set_ylabel('Imaginary Part')
    ax.set_title('Pole-Zero Plot')
    ax.add_artist(circle1)
        
    return ax

Some of the plots add lines connecting poles and zeros to points of constant frequency on the unit circle.  This function will plot the pole-zero diagram and add these lines.

In [ ]:
def plot_with_lines(z, p, omega, name):
    """
        Draw lines from the poles and zeros to the point on the unit circle.
        Input:
            z: represent zeros, complex number
            p: represent pole, complex number
            omega: frequency to connect lines to
            name: the name of the plot to be saved
    """
    # Generate the pole-zero diagram
    ax = pz_plot(z, p)
    
    # Obtain the coordinate of the constant frequency term
    mx = np.cos(omega)
    my = np.sin(omega)
    
    # Plot lines for the poles
    for i in range(len(p)):
        ax.plot([np.real(p[i]), mx],
                [np.imag(p[i]), my],
                'g')

    # and for the zeros
    for i in range(len(z)):
        ax.plot([np.real(z[i]), mx],
                [np.imag(z[i]), my],
                'r')

    # Save figure in python or ipython system
    if not is_jupyter(): plt.savefig(name)

### Magnitude Response
First, let's plot the magnitude response as a function of frequency

In [ ]:
# Use freqz function to compute frequency response 
# of a digital filter
(w,h) = sps.freqz(b, a)

# calculate magnitude response in dB scale
y = 20 * np.log10(np.abs(h))

plot_response(w, y, ylim = [-40, 10],
              ylabel = 'Magnitude (dB)',
              title = 'Magnitude Response(dB)',
              name = 'Example_5_2_1_magnitude.pdf')

### Phase Response

In [ ]:
# calculate the phase response
phase = np.angle(h)

# phase plot
plot_response(w, phase, ylim = [-1.6, 0],
              ylabel = 'Phase(radians)',
              title = 'Phase Response',
              name = 'Example_5_2_1_phase.pdf')

### Show the pole-zero plot

In [ ]:
# Use tf2zpk function to find the pole, zeros of response
(z,p,k) = sps.tf2zpk(b, a)
print(len(p))
pz_plot(z, p)

# Save figure in python or ipython system
if not is_jupyter(): plt.savefig('Example_5_2_1_pole_zero.pdf')

Draw lines from the poles and zeros to the point on the unit circle corresponding to the frequency of $\omega=\pi/4$;

In [ ]:
plot_with_lines(z = z, p = p, omega = np.pi/4, name = 'Example_5_2_1_pole_zero_pi_4.pdf')

for $\omega=\pi/2$;

In [ ]:
plot_with_lines(z = z, p = p, omega = np.pi/2, name = 'Example_5_2_1_pole_zero_pi_2.pdf')

and finally, for $\omega=3\pi/4$.

In [ ]:
plot_with_lines(z = z, p = p, omega = 3*np.pi/4, name = 'Example_5_2_1_pole_zero_3pi_4.pdf')

© The University of Edinburgh: Produced by D. Laurenson, School of Engineering. Initial code conversion by Xing Zixiao.